In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn


In [3]:
x = torch.ones(2, 10, 5)

In [4]:
from hw_ss.model.SpEX_plus import SpEXPlus

torch.Size([10, 1, 47999])


In [14]:
kw = {
    "L1": 20,
      "L2": 80,
      "L3": 160,
      "speech_encoder_out_channels": 30,
      "extractor_emb_dim": 20,
      "extractor_intermed_dim": 10,
      "num_tcn_blocks_in_stack": 2,
      "num_stacked_tcn": 2,
      "num_res_net_blocks": 2,
      "spk_emb": 100,
      "num_spk": 2,
      "tcn_kernel_size": 3
}

In [16]:
model = SpEXPlus(**kw)

In [17]:
model

SpEXPlus(
  (speech_and_speaker_encoder): SpeechEncoder(
    (encoder_short): Conv1d(1, 30, kernel_size=(20,), stride=(10,))
    (encoder_middle): Conv1d(1, 30, kernel_size=(80,), stride=(10,))
    (encoder_long): Conv1d(1, 30, kernel_size=(160,), stride=(10,))
  )
  (channel_norm): ChannelWiseLayerNorm(
    (norm): LayerNorm((90,), eps=1e-05, elementwise_affine=True)
  )
  (conv1x1_extractor): Conv1d(90, 20, kernel_size=(1,), stride=(1,))
  (tcn_extractors): SpeakerExtractor(
    (extractor): ModuleDict(
      (StackedTCN #1): StackedTCN(
        (STCN): ModuleDict(
          (FTCN): FTCNBlock(
            (block): Sequential(
              (0): Conv1d(120, 10, kernel_size=(1,), stride=(1,))
              (1): PReLU(num_parameters=1)
              (2): GlobalLayerNorm()
              (3): Conv1d(10, 10, kernel_size=(3,), stride=(1,), padding=(1,), groups=10)
              (4): PReLU(num_parameters=1)
              (5): GlobalLayerNorm()
              (6): Conv1d(10, 20, kernel_size=(1

In [21]:
import torch 
import torch.nn as nn
batch = torch.randn(10, 1, 48000)


In [15]:
from torchmetrics.audio.pesq import PerceptualEvaluationSpeechQuality

In [16]:
pesq = PerceptualEvaluationSpeechQuality(16000, "nb")

In [22]:
pesq(batch, torch.ones_like(batch))

tensor(2.3714)

In [22]:
s1, s2, s3, pred = model(batch, batch)

In [23]:
s1.shape

torch.Size([10, 1, 48000])

In [37]:
def SISDRLoss(pred, target):
    pred = pred.squeeze(1)
    target = target.squeeze(1)

    pred_hat = pred - pred.mean(dim=-1).unsqueeze(-1)
    target_hat = target - target.mean(dim=-1).unsqueeze(-1)

    pred_t = torch.inner(pred_hat, target_hat) * target_hat / (target_hat ** 2).sum(dim=-1)
    target_t = pred_hat - pred_t
    
    ret = 20 * torch.log10(torch.norm(pred_t) / (torch.norm(target_t) + 1e-6))
    return ret

In [38]:
SISDRLoss(torch.randn(size=(3, 1, 10)), torch.randn(size=(3, 1, 10)))

AttributeError: 'Tensor' object has no attribute 'unsqeeze'

In [31]:
a = torch.randn(size=(3, 10))
b = torch.randn(size=(3, 10))

In [36]:
(b - b.mean(dim=-1).unsqueeze(-1)).shape

torch.Size([3, 10])

In [106]:
lin = linear_proj(subsampled)

In [107]:
from feed_forward import ConformerFeedForwardLayer

In [108]:
ffl = ConformerFeedForwardLayer(512, 2, 0.1)

In [109]:
after_ffl = ffl(lin) + lin

In [110]:
attn = ConformerAttentionBlock(512, 8, 0.1)

In [111]:
after_attn = attn(after_ffl) + after_ffl

In [112]:
from conv import ConformerConvBlock

In [116]:
conv_ = ConformerConvBlock(in_chanels=512, exp_factor=2, kernel_size=31, padding=15, stride=1, dropout=0.1)

In [117]:
after_attn.shape

torch.Size([3, 31, 512])

In [118]:
after_conv = conv_(after_attn)


torch.Size([3, 1024, 31])
torch.Size([3, 512, 31])
torch.Size([3, 512, 31])
torch.Size([3, 512, 31])
torch.Size([3, 512, 31])
torch.Size([3, 512, 31])
torch.Size([3, 512, 31])


In [93]:
ffl_2 = ConformerFeedForwardLayer(512, 2, 0.1)

In [95]:
shit = (ffl_2(after_conv) + after_conv)

In [96]:
shit

torch.Size([3, 31, 512])

In [176]:
class ConvSubsample(nn.Module):
    def __init__(self, in_chanels, out_chanels):
        super().__init__()
        self.subsampler = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=out_chanels, kernel_size=3, stride=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=out_chanels, out_channels=out_chanels, kernel_size=3, stride=2),
            nn.ReLU()
        )
    
    def forward(self, x):
        out = self.subsampler(x.unsqueeze(1))
        batch_size, channels, subsample_len, subsample_dim = out.shape

        out = out.permute(0, 2, 1, 3)
        out = out.contiguous().view(batch_size, subsample_len, channels * subsample_dim)
        return out
    


In [181]:
testing_jopa = ConvSubsample(1, 144)
linear = nn.Linear(144 * ((140 - 1)//2 - 1)//2, 144)

In [178]:
batch = torch.randn(3, 128, 140)
batch.shape

torch.Size([3, 128, 140])

In [186]:
out = testing_jopa(batch)
linear(out).shape

3 144 31 34
4896


torch.Size([3, 31, 144])

In [184]:
144 * (((140 - 1) // 2 - 1) // 2)

4896

In [25]:
int('/kaggle/working/MixTrain360/1027_6505_000118_0-ref.wav'.split("/")[-1].split("_")[0])

1027

In [26]:
a = ["jopa", "koka"]
for k,v in enumerate(a):
    print(k, v)

0 jopa
1 koka
